In [168]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [212]:
OUTPUT_DIR = '/Users/keefetan/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/Desktop/NUS/BZA/Y3S2/IS3107/IS3107Project/dags/data'
OUTPUT_FILE = 'predictions.csv'

In [214]:
def is_unwanted(val):
    if pd.isnull(val):
        return False
    if isinstance(val, str) and ('.' in val):
        return False
    return True
df = pd.read_csv(os.path.join(OUTPUT_DIR, 'joinedtable.csv'))


In [41]:
df = pd.read_csv(os.path.join(OUTPUT_DIR, 'joinedtable.csv'))
df['next_day_close'] = df.groupby('ticker')['close'].shift(1)
df['price_increase'] = (df['next_day_close'] - df['close'] > 0).astype(int)
df.replace('.', pd.NA, inplace= True)
df.dropna(inplace=True)
df['TreasuryYield'] = df['TreasuryYield'].astype(float)

In [44]:
df['date'] = pd.to_datetime(df['date'])
latest_date = df['date'].max()
test_data = df[df['date'] == latest_date]
training_data = df[df['date'] != latest_date]

In [153]:
training_data

,ticker,date,close,volume,rsi,sma,TreasuryYield,Cpi,EPS_Diff,RealGdpPerCapita,next_day_close,price_increase
10,AAPL,2024-04-17,168.00,50901210,43.9234,170.5730,4.59,312.332,0.08,67513,167.04,0
11,AMZN,2024-04-17,181.28,31359673,54.6038,185.2750,4.59,312.332,0.20,67513,179.22,0
12,JPM,2024-04-17,180.08,9017093,25.8580,191.9640,4.59,312.332,0.33,67513,181.25,1
13,MSFT,2024-04-17,411.84,15855485,44.5948,422.7090,4.59,312.332,0.15,67513,404.27,0
14,V,2024-04-17,272.69,5940897,40.8391,275.5040,4.59,312.332,0.07,67513,271.37,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4820,AAPL,2020-06-19,349.72,66118952,68.6778,84.3521,0.70,257.797,0.08,57383,358.87,1
4821,AMZN,2020-06-19,2675.01,5776963,69.4418,129.9927,0.70,257.797,-0.06,57383,2713.82,1
4822,JPM,2020-06-19,97.81,40322181,49.1033,92.4900,0.70,257.797,-1.06,57383,96.75,0
4823,MSFT,2020-06-19,195.15,44441141,67.9680,185.0736,0.70,257.797,0.14,57383,200.57,1


In [199]:
#Train-test split 
#training_data_encoded = pd.get_dummies(training_data, columns=[training_data.columns[0]], prefix=["col0"])
X = training_data.drop(columns=[training_data.columns[0], training_data.columns[1],training_data.columns[10],training_data.columns[11]])
y = training_data.iloc[:, 11]  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [178]:
# Define model parameters for tuning
model_params = [
    (GradientBoostingClassifier(random_state=42), {
        'classifier__n_estimators': [300, 400, 500],
        'classifier__learning_rate': [0.01, 0.05],
        'classifier__max_depth': [3, 5, 10, 15],
    })
]

best_accuracy = 0
best_classifier = None
best_params = None

for model, params in model_params:
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', model)
    ])
    grid_search = GridSearchCV(pipeline, params, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    
    if grid_search.best_score_ > best_accuracy:
        best_accuracy = grid_search.best_score_
        best_classifier = model.__class__.__name__
        best_params = grid_search.best_params_

# Print the best results
print(f"Best Grid Search Score: {best_accuracy * 100:.2f}%")
print(f"Best Classifier: {best_classifier}")
print(f"Best Parameters: {best_params}")

Best Grid Search Score: 53.01%
Best Classifier: GradientBoostingClassifier
Best Parameters: {'classifier__learning_rate': 0.05, 'classifier__max_depth': 15, 'classifier__n_estimators': 300}


In [179]:
log_reg_params = {
    'penalty': ['l1', 'l2'],
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'saga']
}

# Initialize logistic regression model
log_reg = LogisticRegression(random_state=42)

# Perform grid search
grid_search = GridSearchCV(log_reg, log_reg_params, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Get best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Print the best results
print("Best Grid Search Score:", best_score * 100)
print("Best Parameters:", best_params)

Best Grid Search Score: 52.37970092050782
Best Parameters: {'C': 0.001, 'penalty': 'l2', 'solver': 'saga'}


In [201]:
#predicting for next day's price increase
test_X = test_data.drop(columns=[test_data.columns[0], test_data.columns[1],test_data.columns[10],test_data.columns[11]])
test_y = test_data.iloc[:, 11] 

In [216]:
params = {
    'learning_rate': 0.05,
    'max_depth': 15,
    'n_estimators': 300
}

classifier = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', GradientBoostingClassifier(**params, random_state=42))
])

classifier.fit(X, y)
predictions = classifier.predict(test_X)
test_data['Predictions'] = predictions
test_data.drop(columns=['price_increase'], inplace=True)

/var/folders/j4/wwb4cnyd7tx5tx39r3xp5sch0000gn/T/ipykernel_80258/3552997970.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['Predictions'] = predictions
/var/folders/j4/wwb4cnyd7tx5tx39r3xp5sch0000gn/T/ipykernel_80258/3552997970.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.drop(columns=['price_increase'], inplace=True)


In [217]:
test_data

,ticker,date,close,volume,rsi,sma,TreasuryYield,Cpi,EPS_Diff,RealGdpPerCapita,next_day_close,Predictions
5,AAPL,2024-04-18,167.04,43122903,39.8979,170.347,4.64,312.332,0.08,67513,165.00,0
6,AMZN,2024-04-18,179.22,30723793,47.2730,185.022,4.64,312.332,0.20,67513,174.63,0
7,JPM,2024-04-18,181.25,9557720,25.5139,190.159,4.64,312.332,0.33,67513,185.80,1
8,MSFT,2024-04-18,404.27,21029917,37.6483,421.273,4.64,312.332,0.15,67513,399.12,0
9,V,2024-04-18,271.37,8231849,39.5878,274.925,4.64,312.332,0.07,67513,269.78,1
